# Base Case with Multiple Stock - Duality

In this nb, we solve the cutting optima problem with big order quantity to cut many specs at the same time with many stocks ( stock with same width and nearly same weight)
The condition/constraints as below: 
- `Generate Pattern` that fit in stock choice and sum each FG cut > lower bound demand
- `Filter Pattern` that keep the trim_cost <= 4%
- `Cut Pattern` that cut many stocks with filtered_pattern to minimize the stocks used

### 0. HELPERS

In [1]:
from typing import Dict, Any
import copy

def make_naive_patterns(stocks, finish, MIN_MARGIN):
    """
    Generates patterns of feasible cuts from stock width to meet specified finish widths.

    Parameters:
    stocks (dict): A dictionary where keys are stock identifiers and values are dictionaries
                   with key 'length' representing the length of each stock.

    finish (dict): A dictionary where keys are finish identifiers and values are dictionaries
                   with key 'length' representing the required finish lengths.

    Returns:
    patterns (list): A list of dictionaries, where each dictionary represents a pattern of cuts.
                   Each pattern dictionary contains 'stock' (the stock identifier) and 'cuts'
                   (a dictionary where keys are finish identifiers and the value is the number
                   of cuts from the stock for each finish).
                   
                   Naive pattern with maximum number of cuts of each Finished Goods
                   that is closet to the required need_cut
                   and SUM(widths of FG) smaller Mother Coil width
    """

    patterns = []
    for f in finish:
        feasible = False
        for s in stocks:
            # max number of f that fit on s
            num_cuts_by_width = int((stocks[s]["width"]-MIN_MARGIN) / finish[f]["width"])
            # max number of f that satisfied the need cut WEIGHT BOUND
            num_cuts_by_weight = int((finish[f]["upper_bound"] * stocks[s]["width"] ) / (finish[f]["width"] * stocks[s]['weight']))
            # min of two max will satisfies both
            num_cuts = min(num_cuts_by_width, num_cuts_by_weight)

            # make pattern and add to list of patterns
            if num_cuts > 0:
                feasible = True
                cuts_dict = {key: 0 for key in finish.keys()}
                cuts_dict[f] = num_cuts
                trim_loss = stocks[s]['width'] - sum([finish[f]["width"] * cuts_dict[f] for f in finish.keys()])
                trim_loss_pct = round(trim_loss/stocks[s]['width'] * 100, 3)
                patterns.append({"stock": s, "cuts": cuts_dict, 'trim_loss':trim_loss, "trim_loss_pct": round(trim_loss_pct,2) })

        if not feasible:
            print(f"No feasible pattern was found for Stock {s} and FG {f}")

    return patterns

def create_finish_demand_by_line_fr_naive_pattern(patterns, finish: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Convert demand in KGs to demand in slice on a specific stock
    """

    dump_ls = {}
    for f, finish_info in finish.items():
        try:
            non_zero_min = min([patterns[i]['cuts'][f] for i, _ in enumerate(patterns) if patterns[i]['cuts'][f] != 0])
        except ValueError:
            non_zero_min = 0
        dump_ls[f] = {**finish_info
                            ,"upper_demand_slice": max([patterns[i]['cuts'][f] for i,_ in enumerate(patterns)])
                            ,"demand_slice": non_zero_min }
    
    # Filtering the dictionary to include only items with keys in keys_to_keep
    new_finish_list = {k: v for k, v in dump_ls.items() if v['upper_demand_slice'] > 0}

    return new_finish_list

In [2]:
def filter_stocks_by_width_and_weight(remain_stock, min_width=None, min_weight=None):
    """
    Filter remain_stock, which has the width and min_weight for each overused stock needed to be replaced
    """
    filtered_stocks = {}
    for stock_id, details in remain_stock.items():
        width = details['width']
        weight = details['weight']
        
        if ((min_width is None or width == min_width) and # dam bao trim loss khong bi thay doi lon
            (min_weight is None or weight >= min_weight)):
            filtered_stocks[stock_id] = details

    # Sort stocks by width and weight
    sorted_stocks = dict(
        sorted(
            filtered_stocks.items(),
            key=lambda item: (item[1]['width'], item[1]['weight'])
        )
    )
    
    return sorted_stocks

### 2. SOLVER - with duality for pattern generation

In [3]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD, value

# NEW PATTERN - OBJECT FUNC KO WORK CHO WEIGHT
# GENERATE ON LINE CUT PATTERN - WEIGHT WONT BE CONSIDER, IN ADDITION, FIRST FIT STOCK WILL BE TAKEN
def new_pattern_problem(finish, width_s, ap_upper_bound, demand_duals, MIN_MARGIN):
    prob = LpProblem("NewPatternProblem", LpMaximize)

    # Decision variables - Pattern
    ap = {f: LpVariable(f"ap_{f}", 0, ap_upper_bound[f], cat="Integer") for f in finish.keys()}

    # Objective function
    # maximize marginal_cut:
    prob += lpSum(ap[f] * demand_duals[f] for f in finish.keys()), "MarginalCut"

    # Constraints - subject to stock_width - MIN MARGIN
    prob += lpSum(ap[f] * finish[f]["width"] for f in finish.keys()) <= width_s - MIN_MARGIN, "StockWidth_MinMargin"
    
    # Constraints - subject to trim loss 4% 
    prob += lpSum(ap[f] * finish[f]["width"] for f in finish.keys()) >= 0.96 * width_s , "StockWidth"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    marg_cost = value(prob.objective)
    pattern = {f: int(ap[f].varValue) for f in finish.keys()}
    return marg_cost, pattern

# DUALITY
def generate_pattern_dual(stocks, finish, patterns, MIN_MARGIN):
    prob = LpProblem("GeneratePatternDual", LpMinimize)

    # Sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    s = {p: patterns[p]["stock"] for p in range(len(patterns))}
    a = {(f, p): patterns[p]["cuts"][f] for p in P for f in F}
    demand_finish = {f: finish[f]["demand_slice"] for f in F}

    # Decision variables #relaxed integrality
    x = {p: LpVariable(f"x_{p}", 0, None, cat="Continuous") for p in P}

    # OBJECTIVE function minimize stock used:
    prob += lpSum(x[p] for p in P), "Cost"

    # Constraints
    for f in F:
        prob += lpSum(a[f, p] * x[p] for p in P) >= demand_finish[f], f"Demand_{f}"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    # Extract dual values
    dual_values = {f: prob.constraints[f"Demand_{f}"].pi for f in F}

    ap_upper_bound = {f: max([patterns[i]['cuts'][f] for i,_ in enumerate(patterns)]) for f in finish.keys()}
    demand_duals = {f: dual_values[f] for f in F}

    marginal_values = {}
    pattern = {}
    for s in stocks.keys():
        marginal_values[s], pattern[s] = new_pattern_problem( #new pattern by line cut (trimloss), ignore weight
            finish, stocks[s]["width"], ap_upper_bound, demand_duals, MIN_MARGIN
        )

    s = max(marginal_values, key=marginal_values.get) # pick the first stock if having same width
    new_pattern = {"stock": s, "cuts": pattern[s]}
    return new_pattern

# CUT KNOWN WEIGHT PATTERNS
def cut_weight_patterns(stocks, finish, patterns):
    # Parameters - unit weight
    c = {p: stocks[pattern['stock']]["weight"]/stocks[pattern['stock']]["width"] for p, pattern in enumerate(patterns)}

    # Create a LP minimization problem
    prob = LpProblem("PatternCuttingProblem", LpMinimize)

    # Define variables
    # x = {p: LpVariable(f"x_{p}", lowBound=0, cat='Integer') for p in range(len(patterns))}
    x = {p: LpVariable(f"x_{p}", 0, 1, cat='Integer') for p in range(len(patterns))} # tu tach ta stock dung nhieu lan thanh 2 3 dong

    # Objective function: minimize total stock use
    prob += lpSum(x[p]* c[p] for p in range(len(patterns))), "TotalCost"

    # Constraints: meet demand for each finished part
    for f in finish:
        prob += lpSum(patterns[p]['cuts'][f] * finish[f]['width'] * x[p] * c[p] for p in range(len(patterns))) >= finish[f]['need_cut'], f"DemandWeight{f}"

    # Solve the problem
    prob.solve()

    # Extract results
    # Fix integer
    solution = [1 if (x[p].varValue > 0 and round(x[p].varValue)==0) else round(x[p].varValue) for p in range(len(patterns))]
    total_cost = sum(solution)

    return solution, total_cost

In [4]:
# COUNT FINAL CUT
def count_weight(data):
    # Initialize an empty dictionary for the total sums
    total_sums = {}
    
    # Loop through each dictionary in the list
    for entry in data:
        count = entry['count']
        cuts = entry['cut_w']
        
        # Loop through each key in the cuts dictionary
        for key, value in cuts.items():
            # If the key is not already in the total_sums dictionary, initialize it to 0
            if key not in total_sums:
                total_sums[key] = 0
            # Add the product of count and value to the corresponding key in the total_sums dictionary
            total_sums[key] += round(count * value,2)
    return total_sums

# Remove stock that produce too many patterns over order
def count_pattern(stock_list):
    """
    Count each stock is used how many times
    """
    
    stock_counts = {}

    # Iterate through the list and count occurrences of each stock
    for item in stock_list:
        stock = item['stock']
        count = 1
        if stock in stock_counts:
            stock_counts[stock] += count
        else:
            stock_counts[stock] = count
            
    return stock_counts

In [5]:
# FILTER STOCK OR PATTERN
def filter_out_stock_by_cond(stock_list, key):
    """
    Find pattern {stock, cuts {}, trim_loss, trim_loss_pct} 
    with condition, take the list of pattern diff from the key
    """
    filtered_list = {}
    for s, stock_info in stock_list.items():
        if s != key:
            filtered_list[s] = {**stock_info}
    
    return filtered_list

def filter_out_pattern_by_cond(pattern_list, cond, key):
    """
    Find pattern {stock, cuts {}, trim_loss, trim_loss_pct} 
    with condition, take the list of pattern diff from the key
    """
    filtered_list = []
    for item in pattern_list:
        if item[cond] != key:
            filtered_list.append(item)
    
    return filtered_list

def filter_out_pattern_by_conds(pattern_list, cond1, cond2, key1,key2):
    """
    Find pattern {stock, cuts {}, trim_loss, trim_loss_pct} 
    with condition, take the list of pattern diff from the key
    """
    filtered_list = []
    for item in pattern_list:
        if item[cond1] == key1 or item[cond2] != key2: # khac stock overused, va giu id co trim loss thap
            filtered_list.append(item)

    # # Sort pattern by trim loss - largest to smallest
    # sorted_pattern = sorted(filtered_list,key=lambda x: x['trim_loss_pct'], reverse=True)
    
    return filtered_list

********

### 3. SOLVING
#### >> Now try combination that can solve at once large qty order
#### >> NO.1 DYNAMIC - Stock with different weight and width

In [6]:
## PARAMETER - CASE 0

PARAMS = {"warehouse": "HSC"
          ,"spec_name": "JSH590R-PO" # yeu cau chuan hoa du lieu OP - PO
          ,"thickness": 2
          ,"maker" : "CSC"
          ,"stock_ratio": { #get from app 
                    "limited": None
                    # "default": 2
                    # "user_setting": 4
                }
        #   ,"forecast_scenario": median
          }

margin_dict = { "HSC":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2.0,
        "margin": 6
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    },
    "NQS":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2.0,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    }
}

# GET ALL PARAMS
MIN_MARGIN = margin_dict[PARAMS["warehouse"]][f"thickness_{PARAMS["thickness"]}"]["margin"]
print(f"MIN_MARGIN:{MIN_MARGIN}")

BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_VALUE:{BOUND_VALUE}")

MIN_MARGIN:8


##### >> LOAD DATA

In [7]:
# Filter stock theo width - weight tang dan - > pick stock thi se chon stock co cung width va weight nhe nhat truoc

# stocks = {
#     'S147': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1080.0,  'weight': 6010,  'qty': 1},
#     'S145': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1080.0,  'weight': 7400,  'qty': 1},
#     'S440': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1135.0,  'weight': 5820,  'qty': 1},
#     'S439': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1135.0,  'weight': 7750,  'qty': 1},
#     'S146': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1188.0,  'weight': 8440,  'qty': 1},
#     'S442': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1135.0,  'weight': 4780,  'qty': 1},
#     'S144': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1188.0,  'weight': 7390,  'qty': 1},
#     'S438': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 5180, 'qty': 1},
#     'S148': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 7390,  'qty': 1},
#         }

In [9]:
finish = {
    'F88': {'width': 390.0, 'need_cut': 2036, 'upper_bound': 3265.795,'fc1': 2432.65,  'fc2': 2930.2,  'fc3': 2706.1},
    'F87': {'width': 354.0,  'need_cut': 9642,  'upper_bound': 13399.847840000002,  'fc1': 15859.4928,  'fc2': 19600.1584,  'fc3': 16639.6624},
    'F86': {'width': 269.0,  'need_cut': 8500,  'upper_bound': 13470.692319999998,  'fc1': 16568.9744,  'fc2': 13670.539200000003,  'fc3': 10857.32},
    'F84': {'width': 215.0,  'need_cut': 4300,  'upper_bound': 6582.8426,  'fc1': 4276.142,  'fc2': 4074.3419999999996, 'fc3': 3594.0579999999995},
    'F91': {'width': 70.0,  'need_cut': 1200,  'upper_bound': 1850,  'fc1': 1006.4503999999997,  'fc2': 1732.5911999999996,  'fc3': 1603.0631999999998}
        }

#### >> GENERATE PATTERN BY DUALITY

In [10]:
# PHASE 1: Generate naive/dual patterns using dual method
print("PHASE 1: Naive/ Dual Pattern Generation")
nai_patterns = make_naive_patterns(stocks, finish, MIN_MARGIN)
len_stocks = len(stocks)
print(f">> Number of stocks: {len_stocks},\n Total Naive patterns: {len(nai_patterns)}")

total_need_cut = sum(item['need_cut'] for item in finish.values())
print(f'>> Total Need cut: {total_need_cut} KGs')

PHASE 1: Naive/ Dual Pattern Generation
>> Number of stocks: 11,
 Total Naive patterns: 55
>> Total Need cut: 25678 KGs


In [11]:
print("PHASE 2: Pattern Generation Duality", end=".")
total_dual_pat = []
# Phase 1.1: Dual Pattern 
# patterns = copy.deepcopy(nai_patterns)  # deep copy
dual_stocks = copy.deepcopy(stocks)
i = 0
rm_stock = True
while rm_stock == True:
    try:
        # patterns = filter_out_pattern_by_cond(patterns, "stock", key = max_key)
        dual_stocks = filter_out_stock_by_cond(dual_stocks,max_key)
        # print(f"remove stock key {max_key}")
    except NameError:
        # print('->> max_key error, next')
        pass
    patterns = make_naive_patterns(dual_stocks, finish, MIN_MARGIN)
    new_finish_list = create_finish_demand_by_line_fr_naive_pattern(patterns, finish)
    new_pattern = generate_pattern_dual(dual_stocks, new_finish_list, patterns, MIN_MARGIN) # Stock nao do toi uu hon stock khac o width thi new pattern luon bi chon cho stock do #FIX
    dual_pat = []
    while new_pattern not in dual_pat:
        patterns.append(new_pattern)        # pattern de generate them new pattern
        total_dual_pat.append(new_pattern)  # tinh tong dual pattern co the duoc generate
        dual_pat.append(new_pattern)        # dual pat de tinh stock bi lap nhieu lan
        new_pattern = generate_pattern_dual(dual_stocks, new_finish_list, patterns, MIN_MARGIN)
        print(end=".")

    # filter stock having too many patterns
    ls = count_pattern(dual_pat)
    max_key = max(ls, key=ls.get)
    max_pat = ls[max_key]
    if max_pat > 1 and i < len_stocks-2:
        rm_stock = True
        i +=1
        print(f"{i} round")
    else: 
        rm_stock = False

# Combine pattterns
sum_patterns = nai_patterns + total_dual_pat

print()
print(f'>> Total naive/dual patterns: {len(sum_patterns)}')

PHASE 2: Pattern Generation Duality......1 round
.....2 round
.....3 round
........4 round
........5 round
.......6 round
.......7 round
.......8 round
.....9 round
.....
>> Total naive/dual patterns: 117


In [12]:
# Phrase 3: Filter Patterns having trim loss as requirements
print("PHASE 3: Filter Patterns", end=":")
filtered_trimloss_pattern = []
idx=0
for pattern in sum_patterns:
    cuts_dict= pattern['cuts']
    width_s = stocks[pattern['stock']]['width']
    trim_loss = width_s - sum([finish[f]["width"] * cuts_dict[f] for f in new_finish_list.keys()])
    trim_loss_pct = round(trim_loss/width_s * 100, 3)
    if trim_loss_pct <= 4.00: 
        idx +=1
        pattern.update({'trim_loss':trim_loss, "trim_loss_pct": round(trim_loss_pct,2), "patt_id":idx})
        filtered_trimloss_pattern.append(pattern)
        print(end=".")
print()
print(f">> The Patterns with less 4% trim loss: {len(filtered_trimloss_pattern)}")

PHASE 3: Filter Patterns:.................................................................
>> The Patterns with less 4% trim loss: 65


In [13]:
# Phrase 4: Cut Pattern to map all pattern to demand requirements
print("PHASE 4: Cut Patterns", end=".")
print()
filtered_stocks = [filtered_trimloss_pattern[i]['stock'] for i in range(len(filtered_trimloss_pattern))]
chosen_stocks = {}
for stock_name, stock_info in stocks.items():
    if stock_name in filtered_stocks:
        chosen_stocks[stock_name]= {**stock_info}

# CHU Y: co truong hop CHOSEN stock < total stock need to be cut
print(f">> Stocks with fit patterns: {chosen_stocks.keys()}") # 

PHASE 4: Cut Patterns.
>> Stocks with fit patterns: dict_keys(['S438', 'S144', 'S148', 'S145', 'S146', 'S441', 'S81', 'S80', 'S439', 'S143', 'S149'])


#### >> START THE LOOP

In [14]:
stt = True
while stt == True:
    try:
        for overused_item in overused_list:
            # filtered_trimloss_pattern {stock,cuts,trim_loss,trim_loss_pct,patt_id}
            filtered_trimloss_pattern = filter_out_pattern_by_conds(filtered_trimloss_pattern, 
                                                                'patt_id', 'stock',
                                                                overused_item['patt_id'] ,overused_item['stock'])
    except NameError:
        pass
    
    # CUT PATTERN
    solution, total_cost = cut_weight_patterns(chosen_stocks, new_finish_list, filtered_trimloss_pattern)
    solution_list = []
    print(f"Total stock need: {total_cost}")
    for i, pattern_info in enumerate(filtered_trimloss_pattern):
        count = solution[i]
        if count > 0.00:
            rcound = round(count)
            if rcound < 1:
                rcound = 1
            cut_pattern = pattern_info['cuts']
            s = pattern_info['stock']
            print(f"+ Take ({rcound}), {pattern_info}")
            sol = {"count": rcound, **pattern_info}
            solution_list.append(sol)
            print()
    
    # Neu lap stock thi rm all pattern tru cai trim loss thap nhat va chay lai
    sorted_solution_list = sorted(solution_list,key=lambda x: (x['stock'],x['trim_loss_pct']))

    # now take first overused stock pattern only
    # first_stock = sorted_solution_list[0]['stock']
    overused_list =[]
    for i, solution_pattern in enumerate(sorted_solution_list):
        if i == 0:
            take_stock = sorted_solution_list[0]['stock']
        else:
            if sorted_solution_list[i]['stock'] == take_stock:
                overused_list.append({'stock':sorted_solution_list[i]['stock'], "patt_id":sorted_solution_list[i]['patt_id'] })
                # take patt_id cua i-1 de giu lai, luu ten stock vao stock lap
            else:
                take_stock = sorted_solution_list[i]['stock']

    if not overused_list:
        stt = False

    

Total stock need: 4
+ Take (1), {'stock': 'S148', 'cuts': {'F88': 0, 'F87': 3, 'F86': 0, 'F84': 0, 'F91': 2}, 'trim_loss': 21.0, 'trim_loss_pct': 1.72, 'patt_id': 12}

+ Take (1), {'stock': 'S148', 'cuts': {'F88': 0, 'F87': 0, 'F86': 2, 'F84': 2, 'F91': 3}, 'trim_loss': 45.0, 'trim_loss_pct': 3.68, 'patt_id': 13}

+ Take (1), {'stock': 'S146', 'cuts': {'F88': 0, 'F87': 2, 'F86': 0, 'F84': 2, 'F91': 1}, 'trim_loss': 11.0, 'trim_loss_pct': 0.9, 'patt_id': 16}

+ Take (1), {'stock': 'S143', 'cuts': {'F88': 1, 'F87': 0, 'F86': 3, 'F84': 0, 'F91': 0}, 'trim_loss': 22.0, 'trim_loss_pct': 1.8, 'patt_id': 33}

Total stock need: 4
+ Take (1), {'stock': 'S148', 'cuts': {'F88': 0, 'F87': 0, 'F86': 2, 'F84': 2, 'F91': 3}, 'trim_loss': 45.0, 'trim_loss_pct': 3.68, 'patt_id': 13}

+ Take (1), {'stock': 'S146', 'cuts': {'F88': 0, 'F87': 2, 'F86': 0, 'F84': 2, 'F91': 1}, 'trim_loss': 11.0, 'trim_loss_pct': 0.9, 'patt_id': 16}

+ Take (1), {'stock': 'S439', 'cuts': {'F88': 0, 'F87': 3, 'F86': 0, 'F84':

#### >> PHASE 5: EVALUATION

In [15]:
# Phase 5: Evaluation Over-cut / Stock Ratio
print("PHASE 5: Evaluation Stock Ratio", end=".")
for i, sol in enumerate(solution_list):
    s = solution_list[i]['stock'] # stock cut
    cuts_dict = solution_list[i]['cuts']
    weight_dict = {f: round(cuts_dict[f]*finish[f]['width']*stocks[s]['weight']/stocks[s]['width'],3) for f in cuts_dict.keys()}
    # print(weight_dict)
    solution_list[i] = {**sol,
                        "cut_w": weight_dict}
print()
# Total Cuts
total_sums = count_weight(solution_list)
print(f">> Total cut of each Finished Goods: {total_sums}")
# MC WEIGHT PER UNIT
over_cut = {}
over_cut_ratio = {}
# Overcut cutweight (slice * fg_width * wu) - need_cut 
for key in total_sums.keys():
    over_cut[key] = round(total_sums[key] - finish[key]['need_cut'],3)
    over_cut_ratio[key] = round(over_cut[key]/finish[key]['fc1'],4)

print(f">> Over-cut: {over_cut}")
print(f">> Stock-ratio on next month forecast: {over_cut_ratio}")


PHASE 5: Evaluation Stock Ratio.
>> Total cut of each Finished Goods: {'F88': 2581.87, 'F87': 11073.03, 'F86': 8593.369999999999, 'F84': 5222.73, 'F91': 2586.2400000000002}
>> Over-cut: {'F88': 545.87, 'F87': 1431.03, 'F86': 93.37, 'F84': 922.73, 'F91': 1386.24}
>> Stock-ratio on next month forecast: {'F88': 0.2244, 'F87': 0.0902, 'F86': 0.0056, 'F84': 0.2158, 'F91': 1.3774}
